In [1]:
# Import libraries
library(shiny)
library(shinythemes)
library(data.table)
library(RCurl)
library(randomForest)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.



In [2]:
# Convert categorical variables to factors
weather$outlook <- as.factor(weather$outlook)
weather$windy <- as.factor(weather$windy)
weather$play <- as.factor(weather$play)

# Check class distribution
table(weather$play)

# Build the Random Forest model
model <- randomForest(play ~ ., data = weather, ntree = 500, mtry = 2, importance = TRUE)

# Print model summary
print(model)

# Check variable importance
print(importance(model))

ERROR: Error: object 'weather' not found


In [5]:
# Define UI
ui <- fluidPage(theme = shinytheme("united"),
  headerPanel('Should you Play Golf?'),
  sidebarPanel(
    HTML("<h3>Input parameters</h3>"),
    selectInput("outlook", "Outlook:", 
                choices = list("Sunny" = "sunny", "Overcast" = "overcast", "Rainy" = "rainy"), 
                selected = "Rainy"),
    sliderInput("temperature", "Temperature:", min = 64, max = 86, value = 70),
    sliderInput("humidity", "Humidity:", min = 65, max = 96, value = 90),
    selectInput("windy", "Windy:", choices = list("Yes" = "TRUE", "No" = "FALSE"), selected = "TRUE"),
    actionButton("submitbutton", "Submit", class = "btn btn-primary")
  ),
  mainPanel(
    tags$label(h3('Status/Output')),
    verbatimTextOutput('contents'),
    tableOutput('tabledata')
  )
)

# Define server
server <- function(input, output, session) {
  datasetInput <- reactive({
    df <- data.frame(
      outlook = input$outlook,
      temperature = input$temperature,
      humidity = input$humidity,
      windy = input$windy,
      stringsAsFactors = FALSE
    )
    df$outlook <- factor(df$outlook, levels = levels(weather$outlook))
    df$windy <- factor(df$windy, levels = levels(weather$windy))
    prediction <- predict(model, df)
    probabilities <- round(predict(model, df, type = "prob"), 3)
    Output <- data.frame(Prediction = prediction, Probabilities = probabilities)
    return(Output)
  })
  output$contents <- renderPrint({
    if (input$submitbutton > 0) {
      isolate("Calculation complete.")
    } else {
      return("Server is ready for calculation.")
    }
  })
  output$tabledata <- renderTable({
    if (input$submitbutton > 0) {
      isolate(datasetInput())
    } 
  })
}

In [8]:
server <- function(input, output, session) {

  # Reactive dataset based on user inputs
  datasetInput <- reactive({
    # Create input data frame
    df <- data.frame(
      outlook = input$outlook,
      temperature = input$temperature,
      humidity = input$humidity,
      windy = input$windy,
      stringsAsFactors = FALSE
    )
    # Align factor levels with training data
    df$outlook <- factor(df$outlook, levels = levels(weather$outlook))
    df$windy <- factor(df$windy, levels = levels(weather$windy))
    # Make predictions
    prediction <- predict(model, df)
    probabilities <- round(predict(model, df, type = "prob"), 3)
    # Combine predictions and probabilities
    Output <- data.frame(Prediction = prediction, Probabilities = probabilities)
    return(Output)
  })
  # Status/Output Text Box
  output$contents <- renderPrint({
    if (input$submitbutton > 0) { 
      isolate("Calculation complete.") 
    } else {
      return("Server is ready for calculation.")
    }
  })
  # Prediction results table
  output$tabledata <- renderTable({
    if (input$submitbutton > 0) {
      isolate(datasetInput())
    }
  })
}


In [ ]:
# Create the shiny app 
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3868



: 